**Objetivo**

Optimizar el tiempo de carrera teorico para coches Mercedes en condiciones de clima seco, 
generando una estratégia de carrera reduciendo el tiempo total de carrera en un 1.5% analizando el comportamiento de los neumaticos. 


1. Seleccionar las carreras con tiempo seco entre las temporadas 2019-2021. Datos historicos reales
2. Parametros
    * Traction
    * Asphalt grip --> si
    * Lateral --> si
    * Braking
    * Asphalt Abrasion --> si
    * Tyre Stress --> si
    * Downforce --> si
    * Car-weight ?
    * pit stop time loss
    * In-lap lost
    * Out-lap lost

    * max_tyre_life_s se calcula a partir de simular todas las carreras

    
    
3. Variables
    NOta: hay 5 compuestos y se deben usar al menos dos componentes diferentes, no hay limite del número total de componentes a usar
    TODO: Identificar el equivalente en cada carrera

    sea c = número de compuesto
    sea s = número de stint
    sea i = vuelta actual en el stint
    sea t = número total de stint "compuestos utilizados"

    tiempo_vuelta_c_i
    número_de_vueltas_s_c
    tiempo_stint_s_c = suma de 1 a número_de_vueltas_s(tiempo_vuelta_c_i)

4. Función Objetivo:
    tiempo_de_carrera = sumatoria desde s = 1 hasta t(tiempo_stint_s_c)
    Miminizar tiempo_de_Carrera 

5. Restricciones
    De acuerdo a FIA regulations 
    1 <c < 5 --> compuestos disponibles
    2 <s > infinito --> número de stint
    1 <i > número total de vueltas para el circuito a analizar PENDIENTE
    2 < t > infinito ---> total de stints
    1 < número_de_vueltas_s_c < max_tyre_life_s

    tiempo_total_carrera = suma(tiempo_vuelta_ci*)

6. Calibración

    Despues de limpiar datos mirar cuales afectan

    
    soft 15 medium 25 soft 12
7. Otras consideraciones
    Restricción: Solo se evaluaran las carreras con tiempo seco durante esos años para Mercerdes (2 Automoviles)
    Solo carreras sin vsc o sc
    NOta: laps["trackstatus"] --> 1 clear --> 2 y flag --> 3 >> 4 full sc -> red flag -> vsc -> vsc ending 
    



In [1]:
import matplotlib.pyplot as plt
import pandas as pd
from timple.timedelta import strftimedelta
import fastf1
import fastf1.plotting
from fastf1.core import Laps
import numpy as np


fastf1.Cache.enable_cache('./')  # replace with your cache directory

# we only want support for timedelta plotting in this example
fastf1.plotting.setup_mpl(mpl_timedelta_support=True, color_scheme=None, misc_mpl_mods=False)
t_races_by_year = [21,17,22]

# exclude vsc and sc races https://f1.fandom.com/wiki/Safety_Car#Safety_Car

#Leer  todas las carreras,
# 1. if any lap has track_status!1 race should not be added
# 2. if any driver dnf the race should not be added
# 3. if weather data has any register with Rain == true should not be added.
#session = fastf1.get_session(2020, 3, 'R')
#session.load()
#race = session



In [2]:
carreras = pd.read_csv('./carreras.csv')
data = pd.DataFrame()
carreras.head()
    

,RaceId,Year,Event,KM per Lap,Laps,Km Total,Weight_loss_lap,Traction,Braking,Lateral,Tyre Stress,Asphalt Grip,Asphalt Abrasion,Downforce,Soft,Medium,Hard
0,1,2019,Abu Dhabi Grand Prix,5.554,54,299.916,1.814815,3,3,3,3,2,3,3,C5,C4,C3
1,2,2019,Australian Grand Prix,5.303,57,302.271,1.719298,3,3,3,3,2,2,3,C4,C3,C2
2,3,2019,Austrian Grand Prix,4.318,70,302.260,1.400000,3,4,3,2,2,1,3,C4,C3,C2
3,4,2019,Hungarian Grand Prix,4.381,68,297.908,1.441176,3,2,4,3,3,2,5,C4,C3,C2
4,5,2019,United States Grand Prix,5.513,55,303.215,1.781818,3,3,4,4,3,4,3,C4,C3,C2


In [105]:
#Obtener los datos de las carreras seleccionadas
column_names = [ 'LapTime', 'LapNumber', 'Stint',  'Compound', 'TyreLife', 'Team', 'Driver',
       'EventName', 'Year']
unused  = ['PitOutTime', 'PitInTime', 'DriverNumber','Time','Sector1Time', 'Sector2Time', 
           'Sector3Time','IsAccurate',
       'Sector1SessionTime', 'Sector2SessionTime',
           'Sector3SessionTime','LapStartTime',
       'SpeedI1', 'SpeedI2', 
        'SpeedFL', 'SpeedST', 
        'IsPersonalBest','LapStartDate',  
           'FreshTyre', 'TrackStatus']

data = pd.DataFrame()
#len(carreras["Event"])
CAR_WEIGHT = 862 # kg
for x in range(len(carreras["Event"])):
    
    year = carreras["Year"][x]
    evento = carreras["Event"][x]
    C_SOFT = carreras["Soft"][x]
    C_MEDIUM = carreras["Medium"][x]
    C_HARD = carreras["Hard"][x]
    race = fastf1.get_session(year, evento, 'R')
    race.load()
    
    componente = []
    laps = race.laps
    n = len(laps["LapTime"])
    
    #Agregar parametros 
    traction = carreras["Traction"][x]
    Traction = [traction for i in range(n)]
    
    braking =  carreras["Braking"][x]
    Braking = [braking for i in range(n)]
    
    lateral = carreras["Lateral"][x]
    Lateral = [lateral for i in range(n)]
    
    tyre_stress = carreras["Tyre Stress"][x]
    TyreStress = [tyre_stress for i in range(n)]
    
    asphaltGrip = carreras["Asphalt Grip"][x]
    AsphaltGrip = [asphaltGrip for i in range(n)]
    
    asphaltAbrasion = carreras["Asphalt Abrasion"][x]
    AsphaltAbrasion = [asphaltGrip for i in range(n)]
    
    downforce = carreras["Downforce"][x]
    Downforce = [downforce for i in range(n)]
    
    km_per_lap = carreras["KM per Lap"][x] 
    KM_Lap = [km_per_lap for i in range(n)]
    
    
    current_weight = []
    weight_loss_per_lap = carreras["Weight_loss_lap"][x]
            
    for i in range(n):
        #Asignar número de componente
        compound = laps["Compound"][i]
        
        if compound == "SOFT":
            componente.append(C_SOFT)
        elif compound == "MEDIUM":
            componente.append(C_MEDIUM)
        elif compound == "HARD":
            componente.append(C_HARD)
        else:
            componente.append("")
        
        #Calcular peso actual del automovil
        lap_number = laps["LapNumber"][i]
        current_weight.append(CAR_WEIGHT - lap_number*weight_loss_per_lap)

    #Agrupar vueltas con mismo evento y number calcular average
    
    #Agregar peso y componente
    laps["CurrentWeight"] = current_weight
    laps["Componente"] = componente
    
    laps["KM_Lap"] = KM_Lap
    
    laps["Traction"] = Traction
    laps["Braking"] = Braking
    laps["Lateral"] = Lateral
    laps["TyreStress"] = TyreStress
    laps["AsphaltGrip"] = AsphaltGrip
    laps["AsphaltAbrasion"] = AsphaltAbrasion
    laps["Downforce"] = Downforce
    
    #Quitar NA
    laps = laps.dropna(subset=["LapTime"])
    laps = laps.dropna(subset=["LapNumber"])
    laps = laps.dropna(subset=["TyreLife"])
    # Quitar columnas que no se usan
    try:
        for c in unused:
            laps.pop(c)
    except:
        print("columns already removed")
    #Añadir nombre del evento
    
    laps["LapTime"] = pd.to_timedelta(laps['LapTime']) / np.timedelta64(1, 's')
    laps["Event"] = evento 
    n = len(laps["LapTime"])
    
    average = []
    group_lap_number=laps.groupby(['LapNumber'],as_index=False).mean()
    #Calcular y guardar la media de cada vuelta como baseline
    for index, row in laps.iterrows():
        lap_number = row['LapNumber']
        try:
            #lap_number_average = group_lap_number[group_lap_number["LapNumber"] == lap_number]
            av = group_lap_number["LapTime"][lap_number-2]
            average.append(av)
        except:
            print("lapnumber", lap_number)
    #Actualizar datos
    laps["AVG"] = average
    data =[data, laps]
    data = pd.concat(data)
    

core           INFO 	Loading data for Abu Dhabi Grand Prix - Race [v2.2.8]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['44', '33', '16', '77', '5', '23', '11', '4', '26', '55', '3', '27', '7', '20', '8', '99', '63', '10', '88', '18']
core           INFO 	Loading data for Australian Grand Prix - Race [v2.2.8]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing

core           INFO 	Loading data for Styrian Grand Prix - Race [v2.2.8]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['33', '44', '77', '11', '4', '55', '16', '18', '14', '22', '7', '5', '3', '31', '99', '47', '6', '9', '63', '10']


In [106]:
 data.to_csv('bd.csv')

In [88]:
group_lap_number=laps.groupby(['LapNumber'],as_index=False).mean()

In [92]:
group_lap_number["LapTime"][0]

106.2078947368421

In [94]:
laps.head()

,LapTime,LapNumber,Stint,Compound,TyreLife,Team,Driver,CurrentWeight,Componente,Event,AVG
1,102.740,2,1,MEDIUM,5.0,Mercedes,HAM,858.370370,C4,Abu Dhabi Grand Prix,106.207895
2,102.503,3,1,MEDIUM,6.0,Mercedes,HAM,856.555556,C4,Abu Dhabi Grand Prix,105.567350
3,102.916,4,1,MEDIUM,7.0,Mercedes,HAM,854.740741,C4,Abu Dhabi Grand Prix,105.676300
4,103.130,5,1,MEDIUM,8.0,Mercedes,HAM,852.925926,C4,Abu Dhabi Grand Prix,105.913600
5,102.858,6,1,MEDIUM,9.0,Mercedes,HAM,851.111111,C4,Abu Dhabi Grand Prix,106.816900


In [83]:
group_lap_number[group_lap_number["LapNumber"] == lap_number]

KeyError: 'LapNumber'

In [77]:
lap_number_average["LapTime"]

KeyError: -1

In [36]:
test=laps.groupby(['LapNumber'],as_index=False).mean()
lap_number =laps['LapNumber'][2]
lap_time = test[test["LapNumber"] == lap_number]
print("lap number", lap_number)
print(lap_time)
print("average", lap_time["LapTime"][1])

lap number 3
   LapNumber    LapTime  Stint  TyreLife  CurrentWeight
1          3  105.56735   1.05      4.45     856.555556
average 105.56734999999999


In [6]:
data.head

<bound method NDFrame.head of                     LapTime  LapNumber  Stint Compound  TyreLife      Team  \
1    0 days 00:01:42.740000          2      1   MEDIUM       5.0  Mercedes   
2    0 days 00:01:42.503000          3      1   MEDIUM       6.0  Mercedes   
3    0 days 00:01:42.916000          4      1   MEDIUM       7.0  Mercedes   
4    0 days 00:01:43.130000          5      1   MEDIUM       8.0  Mercedes   
5    0 days 00:01:42.858000          6      1   MEDIUM       9.0  Mercedes   
...                     ...        ...    ...      ...       ...       ...   
1290 0 days 00:01:11.512000         32      3     HARD       7.0  Williams   
1291 0 days 00:01:11.009000         33      3     HARD       8.0  Williams   
1292 0 days 00:01:10.878000         34      3     HARD       9.0  Williams   
1293 0 days 00:01:11.073000         35      3     HARD      10.0  Williams   
1294 0 days 00:01:18.351000         36      3     HARD      11.0  Williams   

     Driver TrackStatus  CurrentW

In [18]:
lapnumber = count_lapnumber['LapTime']
print(lapnumber)

LapNumber
2     19
3     20
4     20
5     20
6     20
7     20
8     20
9     20
10    20
11    20
12    20
13    20
14    20
15    20
16    20
17    20
18    20
19    20
20    20
21    20
22    20
23    20
24    20
25    20
26    20
27    20
28    20
29    20
30    20
31    20
32    20
33    20
34    20
35    20
36    20
37    20
38    20
39    20
40    20
41    20
42    20
43    20
44    20
45    20
46    19
47    19
48    19
49    19
50    19
51    19
52    19
53    19
54    17
55     6
Name: LapTime, dtype: int64


In [ ]:
print(type(laps["Compound"][3]))

In [ ]:
data.head()

In [ ]:
#Get all races  between 2019-2021 that complain  with the constrains.
column_names = [  'LapTime', 'LapNumber', 'Stint', 'PitOutTime',
       'PitInTime', 'Compound', 'TyreLife', 'FreshTyre',  'Team', 'Driver',
       'TrackStatus','EventName', 'Year', 'RaceTime'],
unused  = ['DriverNumber','Time','Sector1Time', 'Sector2Time', 'Sector3Time','IsAccurate'
       'Sector1SessionTime', 'Sector2SessionTime', 'Sector3SessionTime','LapStartTime'
       'SpeedI1', 'SpeedI2', 'SpeedFL', 'SpeedST', 'IsPersonalBest','LapStartDate']


data = pd.DataFrame()
for j in range(3):
    t = t_races_by_year[j]
    year = years[j]

    for i in range(1,t+1):
        
        race = fastf1.get_session(year, i, 'R')
        race.load()

        #1Check if the race was wet
        weather_data = race.weather_data
        weather_data = set(weather_data['Rainfall'])
        if (True in weather_data):
            print("1. weather")
            continue
        #2 Check if there was any vsc, sc or red flag
        laps = race.laps
        track_status = set(laps["TrackStatus"])
        if ('4' in track_status or  '5' in track_status or '6' in track_status or '7' in track_status):
            print("2. track not clear")
            continue
        #3 Check if driver finished the race without being lapped.
        
        
        laps = race.laps.pick_drivers(drivers).dropna(subset=["LapTime"])
        #print("before-----------", laps.columns)                                    
        # remove unused collumns
        try:
            for c in unused:
                laps.pop(c)
        except:
            print("columns already removed")

        #print("after-----------,", laps)
        print(year, i, race.event["EventName"])
        laps["Event"] = race.event["EventName"]    
        laps["Year"] = year
        data = [data, laps]
        data = pd.concat(data)
        print("mydata", data)
    data.to_csv('data.csv')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.metrics import confusion_matrix, precision_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
from sklearn.model_selection import cross_val_score,StratifiedKFold,RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix,precision_score,f1_score,recall_score
from sklearn.neural_network import MLPClassifier, MLPRegressor
plt.style.use('seaborn')

np.set_printoptions(precision=4)


In [ ]:
temp = session.get_driver("44")
print(session.event["EventName"])
laps = session.laps.pick_driver("44")
laps["Event"] = session.event["EventName"]    
laps.head(5)
#print("laps",session.laps.pick_driver("44"))
#print("hola", session.get_driver("44"))
#print(temp["Status"])
#print(session.results.head(5))

In [ ]:

drivers =session.drivers

#only the 5 drivers  that completed the race lenght
drivers = drivers[:5]
weather_data = session.weather_data

print(weather_data.head(20))
laps = session.laps
print(laps.head(20))
total_laps = 0
total_race_time = 0

race_laps = laps.pick_drivers(drivers)

total_race_time = (race_laps["LapTime"].sum())
total_laps = len(race_laps["LapTime"])
total_drivers = len(drivers)
print("total_race_time",total_race_time)
print("total_laps", total_laps)
#Average race time
avg_race_time = total_race_time / total_drivers
print("avg_race_time",avg_race_time)

#Average lap time
avg_lap_time = total_race_time / total_laps
print("avg_lap_time", avg_lap_time)

## Average lap time all drivers
avg_lap_time  = laps["LapTime"].mean()
print(avg_lap_time)


## neumaticos
"""

laps = laps.pick_team("Renault")
print(medium_compound)

medium_compound = laps.query("Compound == 'MEDIUM'")
total_race_time_medium_compound = medium_compound["LapTime"].sum()
total_laps_medium_compound = len(medium_compound["LapTime"])
avg_race_time_medium_compound = total_race_time_medium_compound / total_laps_medium_compound
print("avg_race_time_medium_compound" ,avg_race_time_medium_compound)
medium_compound[]
"""
"""
hard_compound = laps.query("Compound == 'HARD'")
total_race_time_hard_compound = hard_compound["LapTime"].sum()
total_laps_hard_compound = len(hard_compound["LapTime"])
avg_race_time_hard_compound = total_race_time_hard_compound / total_laps_hard_compound
print("avg_race_time_hard_compound" ,avg_race_time_hard_compound)
"""

"""
soft_compound = laps.query("Compound == 'SOFT'")
total_race_time_soft_compound = soft_compound["LapTime"].sum()
total_laps_soft_compound = len(soft_compound["LapTime"])
avg_race_time_soft_compound = total_race_time_soft_compound / total_laps_soft_compound
print("avg_race_time_soft_compound" ,avg_race_time_soft_compound)
"""